In [ ]:
import nltk 
nltk.download('punkt')
nltk.download('wordnet')
import numpy as np
import pandas as pd
import string
nltk.download('stopwords')
from sklearn.utils import shuffle

import re
#from gensim import corpora
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer

import csv 
import ast

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/CZ4034 Information Retrieval/Classification/combined_csv.csv')
#data = shuffle(data)

raw_tweets = data.content
tweets_date = data.date
tweets_user = data.user

tweets_location = [] 

for data in tweets_user: 
    loc = ast.literal_eval(data)
    tweets_location.append(loc.get("location"))
    

In [ ]:
print(tweets_user[1])
print(raw_tweets[1])
print(tweets_date[1])
print(tweets_location[1])

{'username': 'AmericaDialogue', 'displayname': 'The American Dialogue', 'id': 247048347, 'description': 'FEBRUARY 24, MMXXI | Responsibility & Accountability | #NeverForget January 6, 2021 | #Healthcare | #COVID19', 'rawDescription': 'FEBRUARY 24, MMXXI | Responsibility & Accountability | #NeverForget January 6, 2021 | #Healthcare | #COVID19', 'descriptionUrls': [], 'verified': False, 'created': '2011-02-04T00:11:20+00:00', 'followersCount': 29153, 'friendsCount': 29256, 'statusesCount': 47216, 'favouritesCount': 2685, 'listedCount': 67, 'mediaCount': 3112, 'location': 'United States of America', 'protected': False, 'linkUrl': 'https://www.the-americandialogue.com/', 'linkTcourl': 'https://t.co/OSAFCIKBgC', 'profileImageUrl': 'https://pbs.twimg.com/profile_images/1364243513974513664/khrOkiDj_normal.jpg', 'profileBannerUrl': 'https://pbs.twimg.com/profile_banners/247048347/1614107467', 'url': 'https://twitter.com/AmericaDialogue'}
.@ABC's broadcast: LIVE: Our @ABC News team has the late

In [ ]:
def clean_texts(data):
    
    #data = data.replace("[^a-zA-Z#]", " ")
    
    # remove any links
    data = re.sub(r'https://\S+', '', data)
    
    # remove hashtags and @usernames
    data = re.sub(r"(#[\d\w\.]+)", '', data)
    data = re.sub(r"(@[\d\w\.]+)", '', data)
    
    
    # casefolding and removing punctuations and special chars
    data = data.lower()
    data = re.sub(r"([^a-zA-Z#\s])", '', data)
    
    # tokenise
    data = word_tokenize(data)

    return data

In [ ]:
clean_tweets = [' '.join(clean_texts(text)) for text in raw_tweets]
print(clean_tweets[0])

if the doesnt approve this emerg use then should not approve it either only effective


In [ ]:
all_stopwords = stopwords.words('english')

# add more words?
stopwo = ['new', 'news', 'broadcast', '.', '\'s', ':'] 
for words in stopwo: 
    all_stopwords.append(words)

In [ ]:
def remove_stopwords(data):
    
    data = word_tokenize(data)
    # remove stopwords
    data = [word for word in data if not word in stopwords.words()]

    return data
 

In [ ]:
cleaner_tweets = [' '.join(remove_stopwords(text)) for text in clean_tweets]


In [ ]:
cleaner_tweets[14]

In [ ]:
lemmatizer = WordNetLemmatizer()
stemmer = SnowballStemmer("english")

def process(text):
    sentence = [] 
    for word in text.split():
        word = lemmatizer.lemmatize(word)
        if not(word.endswith('e')):
                word = stemmer.stem(word)
        sentence.append(word)
    return sentence
    

In [ ]:
processed_tweets = [' '.join(process(text)) for text in cleaner_tweets]
print(processed_tweets[14])

In [ ]:
data = {'Processed': processed_tweets, 
        'Raw': raw_tweets,
        'Date': tweets_date,
        'Location': tweets_location} 
df = pd.DataFrame(data) 

df.to_csv('processed_tweets.csv')

In [ ]:
df.head()
df.tail()

,Processed,Raw,Date,Location
60770,florida governor shock state pull covid vaccin...,https://t.co/M92sw0V2ns Florida Governor shoc...,2021-02-18 20:44:24+00:00,United Kingdom
60771,expect vaccine covid variantscovid forev ever ...,*****NOW WE CAN EXPECT MORE VACCINES FOR THE C...,2021-02-18 20:44:23+00:00,"Wisconsin, USA"
60772,dr b build free nationwide covid standbi list ...,Dr. B is building a free nationwide COVID-19 #...,2021-02-18 20:44:20+00:00,"Portland, OR"
60773,clinic trial indicate india vaccine effective ...,Clinical trials indicate India’s vaccines will...,2021-02-18 20:44:17+00:00,"California, USA"
60774,taken first dose covid vaccine parson say sign...,. @GovParsonMO and @FirstLadyTeresa have taken...,2021-02-18 20:44:06+00:00,"Richmond Heights, MO"


In [ ]:
df_clean = pd.read_csv('/content/drive/MyDrive/combined&processed_tweets.csv',
                 lineterminator='\n')
df_clean.tail()

,Unnamed: 0,Processed,Raw,Date,Location\r
60771,60770,florida governor shock state pull covid vaccin...,https://t.co/M92sw0V2ns Florida Governor shoc...,2021-02-18 20:44:24+00:00,United Kingdom\r
60772,60771,expect vaccine covid variantscovid forev ever ...,*****NOW WE CAN EXPECT MORE VACCINES FOR THE C...,2021-02-18 20:44:23+00:00,"Wisconsin, USA\r"
60773,60772,dr b build free nationwide covid standbi list ...,Dr. B is building a free nationwide COVID-19 #...,2021-02-18 20:44:20+00:00,"Portland, OR\r"
60774,60773,clinic trial indicate india vaccine effective ...,Clinical trials indicate India’s vaccines will...,2021-02-18 20:44:17+00:00,"California, USA\r"
60775,60774,taken first dose covid vaccine parson say sign...,. @GovParsonMO and @FirstLadyTeresa have taken...,2021-02-18 20:44:06+00:00,"Richmond Heights, MO"


In [ ]:
df_clean.shape

(60776, 5)

In [ ]:
df_new = df_clean[df_clean['Processed'].notnull()]
df_new

,Unnamed: 0,Processed,Raw,Date,Location\r
0,0,doesnt approve emerg use approve either effective,@KSL5TV If the @US_FDA doesn’t approve this em...,2021-02-19 23:59:58+00:00,USA 🇺🇸\r
1,1,broadcast live news team latest coronavirus pa...,.@ABC's broadcast: LIVE: Our @ABC News team ha...,2021-02-19 23:59:56+00:00,United States of America\r
2,2,thought covid vaccine lead antibodi induc misc...,Thoughts on the covid vaccines leading to anti...,2021-02-19 23:59:55+00:00,\r
3,3,continue work toward make sure patient access ...,We will continue to work towards making sure t...,2021-02-19 23:59:55+00:00,"Cleveland, OH\r"
4,4,shot arm infect prevent consider covid vaccine...,A Shot In the Arm - Infection prevention consi...,2021-02-19 23:59:44+00:00,"Atlanta, GA\r"
...,...,...,...,...,...
60771,60770,florida governor shock state pull covid vaccin...,https://t.co/M92sw0V2ns Florida Governor shoc...,2021-02-18 20:44:24+00:00,United Kingdom\r
60772,60771,expect vaccine covid variantscovid forev ever ...,*****NOW WE CAN EXPECT MORE VACCINES FOR THE C...,2021-02-18 20:44:23+00:00,"Wisconsin, USA\r"
60773,60772,dr b build free nationwide covid standbi list ...,Dr. B is building a free nationwide COVID-19 #...,2021-02-18 20:44:20+00:00,"Portland, OR\r"
60774,60773,clinic trial indicate india vaccine effective ...,Clinical trials indicate India’s vaccines will...,2021-02-18 20:44:17+00:00,"California, USA\r"


In [ ]:
wordcount = df_new['Raw'].apply(lambda x: len(x.split())).sum()
print("There are {} words in the corpus before any processing.".format(wordcount))

wordcount = df_new['Processed'].apply(lambda x: len(x.split())).sum()
print("There are {} words in the corpus after pre-processing.".format(wordcount))

There are 1453183 words in the corpus before any processing.
There are 772003 words in the corpus after pre-processing.


In [ ]:
df_new.Raw.str.split(expand=True).stack().value_counts()

the                        47234
to                         38990
vaccine                    31621
of                         26328
COVID-19                   19701
                           ...  
https://t.co/3ebvWRSabE        1
https://t.co/EjifzaFbI4        1
https://t.co/r0wJCPkUor        1
woeful                         1
#breatheagain                  1
Length: 159015, dtype: int64

In [ ]:
df_new.Processed.str.split(expand=True).stack().value_counts()

vaccine        58250
covid          37047
coronavirus    14135
get             9328
people          5075
               ...  
kaisr              1
getter             1
sajjan             1
stupidest          1
prasar             1
Length: 30240, dtype: int64